# Data validation
This is a mostly manual section, where we check data validity and inconsistencies. Also, it is important for the data to be resilient to bugs.

In [2]:
import os
import pickle as pkl
import re
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# %matplotlib inline
warnings.filterwarnings('ignore')

In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classi

# set sandbox_mode boolean for image building
* if sandbox_mode == True: faster to run, but images won't be generated

In [3]:
sandbox_mode = True

# define functions

In [4]:
# get numerical columns
def get_numerical_mask(df):
    type_mask = []
    for i in df.dtypes:
        if str(i).startswith('float') or str(i).startswith('int'): # or str(i).startswith('bool')
            type_mask.append(True)
        else: type_mask.append(False)
    num_cols = list(np.array(df.columns)[type_mask])
    other_cols = list(np.array(df.columns)[[not elem for elem in type_mask]])
    
    return num_cols, other_cols

# build boxplots
def build_boxplots(df, file):
    
    plt.close()
    
    cols = df.columns
    fig, axes = plt.subplots(1,len(cols),figsize=(15,5))

    for ax,col in zip(axes, cols):
        sns.boxplot(y = col, data = df[cols], ax = ax, width=.1)
    plt.tight_layout()
    plt.plot()
    
    # save file
    file = '01boxplot_'+file+'.jpg'
    plt.savefig(os.path.join(reports,file), bbox_inches = "tight")

# Define paths and capture data

In [5]:
inputs = os.path.join('..', 'data', '01_raw')
outputs = os.path.join('..', 'data', '02_intermediate')
reports = os.path.join('..', 'data', '06_reporting')

filename = 'train_identity.csv'
data = pd.read_excel(os.path.join(inputs, filename))

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'Transact'

# standardize data
For now I am going to consider that cycle might be an important feature: machine's probability to fail might increase according to how long it is being used.

Also, I'll manually set id.

Further investigation is needed, but it seems a reasonable inference for now.

In [ ]:
data.rename({c: c.lower() for c in data.columns}, axis=1, inplace=True)
data.rename({'fail': 'y'}, axis=1, inplace=True)
data.index.rename('id', inplace=True)

# check duplicates for main data

In [ ]:
def duplicate_cleanup(df, col_id):
    original_size = df.shape[0]
    col_subset = df.columns.to_list()
    n_duplicates = df[(df.duplicated(col_subset))].shape[0]
    df.drop_duplicates(col_subset, inplace=True)
    print('number of removed duplicates:', n_duplicates)
    
    true_duplicates = df[(df.duplicated(col_id))].shape[0]
    if true_duplicates != 0:
        raise ValueError('There are still duplicates to verify')
        
    return df

data = duplicate_cleanup(data, 'cycle')

# build new variables

In [ ]:
for i in range(0,data.shape[0]-3):
    data.loc[data.index[i+3],'lag_1'] = data.loc[i+2,'y']
    data.loc[data.index[i+3],'lag_2'] = data.loc[i+1,'y'] and data.loc[i+2,'y']
    data.loc[data.index[i+3],'lag_3'] = data.loc[i,'y'] and data.loc[i+1,'y'] and data.loc[i+2,'y']
    
# bool -> int
for col in ['lag_1', 'lag_2', 'lag_3']:
    data[col] = data[col]*1
    
for i in range(0,data.shape[0]-1):
#     data.loc[data.index[i],'y_forecast1'] = (data.loc[i,'y'] and data.loc[i+1,'y'])
    data.loc[data.index[i],'y_forecast1'] = data.loc[i+1,'y']

#### get composed preset
I abandoned this transformation as I verified that it is not important

In [ ]:
# def get_comp(preset1, preset2):
#     preset = str(preset1)+str(preset2)
#     preset = int(preset)
#     return preset
# data['preset_comp'] = data.apply(lambda x: get_comp(x['preset_1'], x['preset_2']), axis=1)

#### check nans

In [ ]:
data[data.isnull().any(axis=1)]

#### consider dropping rows because of nan on lag or forecast

In [ ]:
# data.dropna(inplace=True)
data = data[~data['y_forecast1'].isna()]

# drop unused columns

skipped as it is not needed for now

# Checking  for possible anomalies in the database
Here we check for some possible anomalies. If there was one, we should look for solutions such as: contact the team responsible for data engineering or anyone who has a good understanding of the provided data, search for flaws in the data pipeline, or at least document for those anomalies.

Check if there is any oddity in data.

In [ ]:
data.loc[:, ].describe()

# boxplots
This is useful to check for outliers, which will be handled later. Also, some values might be misleading, requiring a preimputation step. For instance: hours per week shouldn't surpass 140. Age shouldn't go well beyond 90.

In [ ]:
numerical_cols, other_cols = get_numerical_mask(data)

In [ ]:
print(numerical_cols)

In [ ]:
partition = int(len(numerical_cols)/1)

In [ ]:
if not sandbox_mode:
    build_boxplots(data[numerical_cols[:partition]], '01')

# measure intervals and frequency

In [ ]:
count = 0
intervals = []
previous = False
for i, row in data.iterrows():
    count += 1
    if row['y'] == True and previous == False:
        intervals.append(count)
        count = 0
    previous = row['y']

In [ ]:
y = data['y']
cumsum = (y * (y.groupby((y != y.shift()).cumsum()).cumcount() + 1)).to_list()

count = 0
frequencies = []
previous = 0
for i, value in enumerate(cumsum):
    if (value == 0 and count > 0):
        frequencies.append(count)
    elif (i == len(cumsum)-1 and value > 0):
        frequencies.append(value)
    count = value

In [ ]:
indexes = data[(data['y'] == True) & (data['lag_1'] == False)].index.to_list()
indexes

In [ ]:
ind_plot = list(range(len(cumsum)))
freq_plot = [0]*len(cumsum)

for i, index in enumerate(indexes):
    ind_plot[index] = indexes[i]
    freq_plot[index] = frequencies[i]

In [ ]:
if not sandbox_mode:
    color = sns.color_palette()
    plt.figure(figsize=(16,4))
    plt.bar(ind_plot, freq_plot, alpha=0.8, color=color[0], width=3)
    # sns.barplot([0,1], [0,1], alpha=0.8, color=color[0])
    plt.ylabel('Number of occurrences', fontsize=12)
    plt.xlabel('Failed', fontsize=12)
    plt.tick_params(
        axis='x', which='both', bottom=False, top=False, labelbottom=False)

    plt.plot()
    plt.savefig(os.path.join(reports,'fail_freq.jpg'), bbox_inches = "tight")

# choose response variable
current y is the 'Failed' variable, but we might decide to change it to y_forecast1, which considers Failures on T+1.

One of those variables must be dropped.

In [ ]:
data['y'] = data['y_forecast1']
data.drop(['y_forecast1'], axis=1, inplace=True)

# save data

In [ ]:
data.to_csv(os.path.join(outputs, 'data.csv'))